#### Prepare Annotations

```

annotation = {
'filename': img_path,
'width': w,
'height': h,
'ann': {
    'bboxes': bboxes.astype(np.float32),
    'labels': labels.astype(np.int64),
    'bboxes_ignore': bboxes_ignore.astype(np.float32),
    'labels_ignore': labels_ignore.astype(np.int64)
}
}

```

In [1]:
annotations = []

In [2]:
! head mmdetection_input.csv

cid,filename,width,height,xmin,ymin,xmax,ymax,num_tokens
122240045,500_F_122240045_e7QHZ8T7WLpsRWP3Ibx6quxaUKXQzs4T.jpg,500,392,29,29,90,50,token_1
209829899,500_F_209829899_lq6d3xclAVkA7XOu3GQKGvxgjqDBdjGF.jpg,500,375,14,14,90,34,token_2
13829594,500_F_13829594_PyEjWU070XPo7RCony9oi14UEi5LEr1T.jpg,500,334,351,14,482,49,token_10
86125167,500_F_86125167_zBSLCTXKbnPenEmP2C7vzyZtwPGkZaYO.jpg,500,334,367,260,481,303,token_5
38550534,500_F_38550534_8VArIDHCnuh8E8je08XTIRahTWeOOuYN.jpg,335,500,20,9,312,43,token_36
123720297,500_F_123720297_dxh673J3iAZVecM0HPbVEMKBfY4BXAN0.jpg,334,500,14,16,141,63,token_10
2754791,500_F_2754791_WgYr12CMnx7HQEfCMeJGYrfKcQD1K7.jpg,334,500,15,11,221,37,token_5
57345528,500_F_57345528_t9edGRRK9Uc0pezgyT34eSTG7JHIRS3B.jpg,500,383,9,7,189,27,token_5
184598300,500_F_184598300_T3DTjPD4RYTtQukmyz3QZzkSyjEyYBas.jpg,350,500,235,31,315,57,token_1


#### Split into test and train

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('mmdetection_input.csv')


In [4]:
df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.95

train = df[msk]
test = df[~msk]

In [5]:
train = train.drop(['split'], axis=1)
test = test.drop(['split'], axis=1)

#### Start from here!!! 

In [6]:
train.to_csv('mmdetection_train.csv', index=None)

In [7]:
test.to_csv('mmdetection_val.csv', index=None)

In [8]:
! wc -l mmdetection_train.csv

12909 mmdetection_train.csv


In [9]:
! wc -l mmdetection_val.csv

663 mmdetection_val.csv


In [35]:
! head -n 5 mmdetection_train.csv

cid,filename,width,height,xmin,ymin,xmax,ymax,num_tokens
122240045,500_F_122240045_e7QHZ8T7WLpsRWP3Ibx6quxaUKXQzs4T.jpg,500,392,29,29,90,50,token_1
209829899,500_F_209829899_lq6d3xclAVkA7XOu3GQKGvxgjqDBdjGF.jpg,500,375,14,14,90,34,token_2
13829594,500_F_13829594_PyEjWU070XPo7RCony9oi14UEi5LEr1T.jpg,500,334,351,14,482,49,token_10
86125167,500_F_86125167_zBSLCTXKbnPenEmP2C7vzyZtwPGkZaYO.jpg,500,334,367,260,481,303,token_5


In [60]:
z =  np.array([class_dict['token_1']]).astype(np.int64)
z

array([1])

In [61]:
class_dict = {'token_1' : 1, 'token_2' : 2, 'token_3' : 3, 'token_5' : 5, 'token_10' : 10, 'token_36' : 36}

def create_annotation(row_dict):
    annotation = {}
    annotation['filename'] = row_dict['filename']
    annotation['width'] = int(row_dict['width'])
    annotation['height'] = int(row_dict['height'])
    annotation['ann'] = {}
    
    bboxes_ignore = np.zeros((0, 4))
    labels_ignore = np.zeros((0, ))
    
    annotation['ann']['bboxes'] = np.array([[
                                             int(row_dict['xmin']),
                                             int(row_dict['ymin']),
                                             int(row_dict['xmax']),
                                             int(row_dict['ymax'])
                                            ]]).astype(np.float32)
    try:
        annotation['ann']['labels'] = np.array([class_dict[row_dict['num_tokens']]]).astype(np.int64)
    except:
        print('WTF')
    annotation['ann']['bboxes_ignore'] = bboxes_ignore.astype(np.float32)
    annotation['ann']['labels_ignore'] = labels_ignore.astype(np.int64)
    return annotation
                         

In [62]:
import csv
import numpy as np


def csv_to_annotation_pkl(csv_file):
    annotations = []
    line_number = 0
    with open(csv_file) as f:
        reader = csv.reader(f, skipinitialspace=True)
        header = next(reader)
        print(header)
        while True:
            try:
                row = next(reader)
                line_number += 1
                if line_number % 1000 == 0:
                    print(line_number)
                row_dict = dict(zip(header, row))
                annotations.append(create_annotation(row_dict))
            except Exception as e:
                break
    return annotations

In [63]:
train_annotations = csv_to_annotation_pkl('mmdetection_train.csv')

['cid', 'filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'num_tokens']
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [64]:
len(train_annotations)

12908

In [65]:
import mmcv
mmcv.dump(train_annotations, '/home/ubuntu/tp/annotations/tp_train.pkl')

In [66]:
test_annotations = csv_to_annotation_pkl('mmdetection_val.csv')

['cid', 'filename', 'width', 'height', 'xmin', 'ymin', 'xmax', 'ymax', 'num_tokens']


In [67]:
mmcv.dump(test_annotations, '/home/ubuntu/tp/annotations/tp_val.pkl')

In [68]:
! du -sh /home/ubuntu/tp/annotations/tp_train.pkl

4.7M	/home/ubuntu/tp/annotations/tp_train.pkl


In [69]:
! du -sh /home/ubuntu/tp/annotations/tp_val.pkl

244K	/home/ubuntu/tp/annotations/tp_val.pkl


### Move train and val datasets to another folder

In [52]:
import os
import shutil
src_file_name = '/home/ubuntu/text-placement/images/' + train.iloc[0]['filename']

In [53]:
def copy_file_to_folder(filename, dest_folder):
    src_file_name = '/home/ubuntu/text-placement/images/' + filename
    if os.path.exists(src_file_name):
        shutil.copy(src_file_name, dest_folder) 
    

In [54]:
copy_file_to_folder(train.iloc[0]['filename'], '/home/ubuntu/tp/mmdet-train')

In [55]:
train['filename'].apply(copy_file_to_folder, args=('/home/ubuntu/tp/mmdet-train',))

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
31       None
         ... 
13538    None
13539    None
13540    None
13541    None
13542    None
13543    None
13544    None
13545    None
13546    None
13547    None
13548    None
13549    None
13550    None
13551    None
13552    None
13554    None
13556    None
13557    None
13558    None
13559    None
13560    None
13561    None
13562    None
13563    None
13564    None
13565    None
13566    None
13567    None
13568    None
13569    None
Name: filename, Length: 12908, dtype: object

In [56]:
! ls -l /home/ubuntu/tp/mmdet-train/ | wc -l

7307


In [57]:
test['filename'].apply(copy_file_to_folder, args=('/home/ubuntu/tp/mmdet-val',))

14       None
15       None
80       None
89       None
121      None
128      None
148      None
151      None
182      None
188      None
199      None
230      None
255      None
267      None
268      None
293      None
307      None
309      None
311      None
315      None
338      None
343      None
364      None
397      None
433      None
447      None
452      None
460      None
472      None
498      None
         ... 
12904    None
12944    None
12985    None
12987    None
12995    None
12997    None
13039    None
13052    None
13059    None
13061    None
13080    None
13100    None
13118    None
13137    None
13227    None
13260    None
13284    None
13285    None
13296    None
13315    None
13350    None
13353    None
13358    None
13378    None
13402    None
13495    None
13507    None
13537    None
13553    None
13555    None
Name: filename, Length: 662, dtype: object

In [58]:
! ls -l /home/ubuntu/tp/mmdet-val/ | wc -l

647
